In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from prosit_t.models import PrositTransformerIntensityPredictor, PrositSimpleIntensityPredictor
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.constants import ALPHABET_UNMOD
from prosit_t.wandb_agent import train_utils
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import keras_nlp
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-07-04 07:45:57.337257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 07:45:57.441903: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-04 07:45:58.073694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-04 07:45:58.073750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [10]:
DEFAULT_CONFIG = {
    "learning_rate": 0.001,
    "batch_size": 64,
    "embedding_output_dim": 64,
    "ff_dim": 32,
    "num_heads": 16,
    "seq_length": 30,
    "len_fion": 6,
    "vocab_dict": ALPHABET_UNMOD,
    "dropout_rate": 0,
    "transformer_dropout": 0.1,
    "dataset": "proteometools",
    "data_source": "/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/input_config.json"
}

In [11]:
project_name = "transforming-prosit"

In [12]:
train_data, val_data = train_utils.get_proteometools_data(DEFAULT_CONFIG)

Optionally Downloading and processing the data...
Annotations directory:  /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/./data/TUM_third_pool
Metadata filepath:  /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/./data/TUM_third_pool_meta_data.parquet
Base directory:  /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks
Reading metadata file from /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/./data/TUM_third_pool_meta_data.parquet
Reading and processing annotation files...
Building annotation dataframe...
Applying metadata filters...
mode:  max
attribute:  peptide_length
mode:  max
attribute:  precursor_charge
Scaling and adding encoded columns...
Extracting feature:  ModificationLocationFeature(pad_to_seq_length=True, padding_element=-1)
Extracting feature:  ModificationLossFeature(pad_to_seq_length=True, padding_element=[0, 0, 0, 0, 0, 0])
Extracting feature:  ModificationGainFeature(pad_to_seq_length=True, padding_element=[0, 0, 0, 0, 0, 0])


2023-07-04 07:53:47.932009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 07:53:48.216047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43648 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:e2:00.0, compute capability: 8.6


In [9]:
# "Training examples", BATCH_SIZE * len(int_data.train_data), "Validation examples", BATCH_SIZE * len(int_data.val_data)

In [ ]:
wandb.init(project=project_name, name="third_pool_2_transformers_block")
cb_wandb = WandbCallback()
callbacks = [cb_wandb]
model = PrositSimpleIntensityPredictor(**DEFAULT_CONFIG)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=DEFAULT_CONFIG["learning_rate"]),
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance],
)
model.fit(
    train_data,
    validation_data=val_data,
    epochs=60,
    callbacks=callbacks,
)

Epoch 1/60
   1/4489 [..............................] - ETA: 1:46:24 - loss: 0.9232 - masked_pearson_correlation_distance: 0.9095

2023-07-04 07:54:46.523745: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


4486/4489 [============================>.] - ETA: 0s - loss: 0.3786 - masked_pearson_correlation_distance: 0.2709

2023-07-04 07:55:22.799371: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230704_075438-aev8ve85/files/model-best/assets


...

4489/4489 [==============================] - 41s 9ms/step - loss: 0.3265 - masked_pearson_correlation_distance: 0.2260 - val_loss: 0.6457 - val_masked_pearson_correlation_distance: 0.5573
Epoch 5/60
4485/4489 [============================>.] - ETA: 0s - loss: 0.3188 - masked_pearson_correlation_distance: 0.2202

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230704_075438-aev8ve85/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230704_075438-aev8ve85/files/model-best/assets
/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230704_075438-aev8ve85/files/model-best)... Done. 0.1s


4489/4489 [==============================] - 42s 9ms/step - loss: 0.3191 - masked_pearson_correlation_distance: 0.2205 - val_loss: 0.6242 - val_masked_pearson_correlation_distance: 0.5290
Epoch 6/60
4483/4489 [============================>.] - ETA: 0s - loss: 0.3105 - masked_pearson_correlation_distance: 0.2143